<a href="https://colab.research.google.com/github/fedesierra613/Inteligencia_Artificial/blob/main/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [226]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn import decomposition
from sklearn.decomposition import PCA, KernelPCA
from sklearn import metrics
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import cross_val_score

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

In [227]:
#Visualizacion de los datos del dataset
data = pd.read_csv(r"heart.csv") 
data.head(-5)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,90.0,1,337,0,38,0,390000.00,0.9,144,0,0,256,0
290,45.0,0,615,1,55,0,222000.00,0.8,141,0,0,257,0
291,60.0,0,320,0,35,0,133000.00,1.4,139,1,0,258,0
292,52.0,0,190,1,38,0,382000.00,1.0,140,1,1,258,0


In [228]:
#Revisar si existen datos nulos o espacios de columnas sin rellenar 
print("Valores nulos en train: \n", data.isna().sum(),"\n")

Valores nulos en train: 
 age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
DEATH_EVENT                 0
dtype: int64 



In [229]:
#Se eliminar de las columnas de características aquellas que no son útiles para nuestro proyecto
data.drop(['time'], axis = 1, inplace = True) 
data.head(-5)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
289,90.0,1,337,0,38,0,390000.00,0.9,144,0,0,0
290,45.0,0,615,1,55,0,222000.00,0.8,141,0,0,0
291,60.0,0,320,0,35,0,133000.00,1.4,139,1,0,0
292,52.0,0,190,1,38,0,382000.00,1.0,140,1,1,0


In [230]:
fig1 = make_subplots(rows=5, cols=3, subplot_titles=data.columns)

for i, col in enumerate(data.columns):
    fig1.add_trace(go.Histogram(x=data[col], name=col), row=(i//3)+1, col=(i%3)+1)
    
fig1.update_layout(height=200*5, showlegend=False)
    
fig1.show()

In [289]:
X = data[['age','anaemia','creatinine_phosphokinase','diabetes','ejection_fraction','high_blood_pressure','platelets','serum_creatinine','serum_sodium','sex','smoking']].values
y = data['DEATH_EVENT'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,test_size=0.25)

In [290]:
scaler = StandardScaler() #Se escalizan los datos
scaler.fit(X_train) #El fit de los datos se hace con el conjunto de entrenamiento!
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test) 

In [291]:
#PCA solamente al conjunto de entrenamiento 
n = 11
pca = decomposition.PCA(n_components=n,whiten=True,svd_solver='auto')
pca.fit(X_train_scaled)
X_train_PCA = pca.transform(X_train_scaled)
X_test_PCA = pca.transform(X_test_scaled)
print("Pesos de PCA:",pca.explained_variance_ratio_)
print("Componentes:",pca.components_)

Pesos de PCA: [0.15224169 0.13211175 0.12555635 0.09786508 0.0881707  0.08532696
 0.07642814 0.07058003 0.06662873 0.05694353 0.04814706]
Componentes: [[ 0.24590246 -0.04349115 -0.06868628 -0.30679713 -0.20115128 -0.14236438
  -0.25484092  0.33391293 -0.18222272  0.57048892  0.49291409]
 [ 0.20790155  0.32094545 -0.24189507  0.28306447 -0.26436156  0.03645558
  -0.11197987  0.47499899 -0.49530005 -0.22769328 -0.32952435]
 [-0.43498787 -0.38032561  0.3975485   0.35414347 -0.44176688 -0.28701955
   0.02948507 -0.03997317 -0.3081458   0.08318589  0.02822104]
 [ 0.14378335 -0.39083431  0.11956605 -0.09088571  0.03594753  0.53594295
   0.60918469  0.28992677 -0.20332471 -0.05244523  0.12742294]
 [ 0.51117627 -0.18080103  0.66311491 -0.08120642  0.11749034 -0.05643785
  -0.27750804  0.10006075  0.07429883  0.00537263 -0.3874696 ]
 [ 0.11709208  0.19944777  0.09012105  0.12393439  0.32433884 -0.6656435
   0.54979694  0.25451525  0.0395652   0.04348366  0.06512002]
 [-0.18458907  0.38147901  0

In [323]:
#Empieza el entrenanmiento

#Regresión logística
log = LogisticRegression(C=0.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=20, solver='liblinear', tol=0.1, verbose=0,
                   warm_start=False)

log.fit(X_train_PCA, y_train)

LogisticRegression(C=0.2, max_iter=200, random_state=20, solver='liblinear',
                   tol=0.1)

In [324]:
pred = log.predict(X_test_PCA)

In [ ]:
print("F1 Score:", metrics.f1_score(y_test, pred,average='macro'))

In [343]:
print("MCC:", metrics.matthews_corrcoef(y_test, pred))

MCC: 0.79011947512032025


In [332]:
#SVM
kernels=['linear', 'poly', 'rbf', 'sigmoid']
#lineal
#Kernel=0
#msv = svm.SVC(kernel=kernels[Kernel])

#polinomial cuadrático
#Kernel=1
#msv = svm.SVC(kernel=kernels[Kernel],degree=2)

#polinomial cúbico
#Kernel=1
#msv = svm.SVC(kernel=kernels[Kernel],degree=3)
#rbf 
Kernel=2
msv = svm.SVC(kernel=kernels[Kernel],C=1)
#https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

msv.fit(X_train_PCA, y_train)

SVC(C=1)

In [333]:
y_test_predicted = msv.predict(X_test_PCA)

In [345]:
print("F1 Score:", metrics.f1_score(y_test, y_test_predicted,average='macro'))

F1 Score: 0.8423529411764706


In [346]:
print("MCC:", metrics.matthews_corrcoef(y_test, y_test_predicted))

MCC: 0.77468516529888065
